In [21]:
import pandas as pd
weather_set_dirty = pd.read_csv('inCSV/Weather_LosAngeles.csv', sep=",")

In [19]:
weather_set_dirty['timestamp'] = pd.to_datetime(weather_set_dirty['timestamp'])
mean_values_by_day = weather_set_dirty.groupby(weather_set_dirty["timestamp"].dt.date).mean(numeric_only=True).round(1)

In [20]:
for col in mean_values_by_day.columns:
    weather_set_dirty[col] = weather_set_dirty.apply(
        lambda row: mean_values_by_day.loc[row['timestamp'].date(), col]
        if pd.isnull(row[col]) else row[col],
        axis=1
    )

In [21]:
weather_set_dirty['cloud_cover_description'].fillna(method='ffill',inplace=True)
#weather_set_dirty.to_csv('outCSV/Clean_Weather_Set.csv', index=False)

In [22]:
weather_set_dirty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77362 entries, 0 to 77361
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   city                     77362 non-null  object        
 1   timestamp                77362 non-null  datetime64[ns]
 2   temperature              77362 non-null  float64       
 3   cloud_cover              77362 non-null  float64       
 4   cloud_cover_description  77362 non-null  object        
 5   pressure                 77362 non-null  float64       
 6   windspeed                77362 non-null  float64       
 7   precipitation            77362 non-null  float64       
 8   felt_temperature         77362 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 5.3+ MB


In [22]:
bike_set_dirty = pd.read_csv('inCSV/metro_2017_2022.csv', sep=',')

In [23]:
bike_set_dirty.sort_values('start_station_id')
bike_set_dirty['start_station_lat'].fillna(method="ffill", inplace=True)
bike_set_dirty['start_station_lon'].fillna(method="ffill", inplace=True)

bike_set_dirty.sort_values('end_station_id')
bike_set_dirty['end_station_lat'].fillna(method="ffill", inplace=True)
bike_set_dirty['end_station_lon'].fillna(method="ffill", inplace=True)

In [24]:
bike_set_dirty_agg_start = bike_set_dirty.groupby('start_station_id')[['start_station_lat', 'start_station_lon']].agg(lambda x: x.mode().iloc[0])
bike_set_dirty_agg_end = bike_set_dirty.groupby('end_station_id')[['end_station_lat', 'end_station_lon']].agg(lambda x: x.mode().iloc[0])

In [25]:
bike_set_dirty_merge = bike_set_dirty.merge(bike_set_dirty_agg_start, on='start_station_id', suffixes=('','_mode'))
bike_set_dirty_merged = bike_set_dirty_merge.merge(bike_set_dirty_agg_end, on='end_station_id', suffixes=('','_mode'))
bike_set_dirty_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 0 to 1552781
Data columns (total 14 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Unnamed: 0              1552782 non-null  int64  
 1   start_time              1552782 non-null  object 
 2   end_time                1552782 non-null  object 
 3   start_station_id        1552782 non-null  int64  
 4   start_station_lat       1552782 non-null  float64
 5   start_station_lon       1552782 non-null  float64
 6   end_station_id          1552782 non-null  int64  
 7   end_station_lat         1552782 non-null  float64
 8   end_station_lon         1552782 non-null  float64
 9   bike_id                 1552782 non-null  object 
 10  start_station_lat_mode  1552782 non-null  float64
 11  start_station_lon_mode  1552782 non-null  float64
 12  end_station_lat_mode    1552782 non-null  float64
 13  end_station_lon_mode    1552782 non-null  float64
dtypes:

In [26]:
for i, row in bike_set_dirty_merged.iterrows():
    bike_set_dirty_merged.at[i, 'start_station_lat'] = bike_set_dirty_merged.loc[i, 'start_station_lat_mode']
    bike_set_dirty_merged.at[i, 'start_station_lon'] = bike_set_dirty_merged.loc[i, 'start_station_lon_mode']
    bike_set_dirty_merged.at[i, 'end_station_lat'] = bike_set_dirty_merged.loc[i, 'end_station_lat_mode']
    bike_set_dirty_merged.at[i, 'end_station_lon'] = bike_set_dirty_merged.loc[i, 'end_station_lon_mode']

In [27]:
bike_set_almost_clean = bike_set_dirty_merged
bike_set_almost_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 0 to 1552781
Data columns (total 14 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Unnamed: 0              1552782 non-null  int64  
 1   start_time              1552782 non-null  object 
 2   end_time                1552782 non-null  object 
 3   start_station_id        1552782 non-null  int64  
 4   start_station_lat       1552782 non-null  float64
 5   start_station_lon       1552782 non-null  float64
 6   end_station_id          1552782 non-null  int64  
 7   end_station_lat         1552782 non-null  float64
 8   end_station_lon         1552782 non-null  float64
 9   bike_id                 1552782 non-null  object 
 10  start_station_lat_mode  1552782 non-null  float64
 11  start_station_lon_mode  1552782 non-null  float64
 12  end_station_lat_mode    1552782 non-null  float64
 13  end_station_lon_mode    1552782 non-null  float64
dtypes:

In [28]:
from datetime import datetime
dateformat0 = "%H/%M/%S %d-%m-%Y"
dateformat1 = "%Y-%m-%d %H:%M:%S"
for i, row in bike_set_almost_clean.iterrows():
    try:
        bike_set_almost_clean.at[i, 'start_time'] = datetime.strptime(bike_set_almost_clean.loc[i, 'start_time'], dateformat0)
    except ValueError:
        bike_set_almost_clean.at[i, 'start_time'] = datetime.strptime(bike_set_almost_clean.loc[i, 'start_time'], dateformat1)
    try:
        bike_set_almost_clean.at[i, 'end_time'] = datetime.strptime(bike_set_almost_clean.loc[i, 'end_time'], dateformat0)
    except ValueError:
        bike_set_almost_clean.at[i, 'end_time'] = datetime.strptime(bike_set_almost_clean.loc[i, 'end_time'], dateformat1)
bike_set_almost_clean.head()

,Unnamed: 0,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id,start_station_lat_mode,start_station_lon_mode,end_station_lat_mode,end_station_lon_mode
0,0,2018-05-29 16:45:00,2018-05-29 17:08:00,3068,34.0532,-118.250954,3014,34.05661,-118.237213,5772,34.0532,-118.250954,34.05661,-118.237213
1,891,2019-08-16 17:12:00,2019-08-16 17:42:00,3068,34.0532,-118.250954,3014,34.05661,-118.237213,16618,34.0532,-118.250954,34.05661,-118.237213
2,1467,2019-01-30 12:22:00,2019-01-30 12:28:00,3068,34.0532,-118.250954,3014,34.05661,-118.237213,06358,34.0532,-118.250954,34.05661,-118.237213
3,6956,2018-05-15 08:23:00,2018-05-15 08:30:00,3068,34.0532,-118.250954,3014,34.05661,-118.237213,6144,34.0532,-118.250954,34.05661,-118.237213
4,11733,2018-11-03 18:30:00,2018-11-03 18:51:00,3068,34.0532,-118.250954,3014,34.05661,-118.237213,5860,34.0532,-118.250954,34.05661,-118.237213


In [29]:
bike_set_clean = bike_set_almost_clean.drop(['start_station_lat_mode', 'start_station_lon_mode', 'end_station_lat_mode', 'end_station_lon_mode'], axis=1)
bike_set_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 0 to 1552781
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Unnamed: 0         1552782 non-null  int64  
 1   start_time         1552782 non-null  object 
 2   end_time           1552782 non-null  object 
 3   start_station_id   1552782 non-null  int64  
 4   start_station_lat  1552782 non-null  float64
 5   start_station_lon  1552782 non-null  float64
 6   end_station_id     1552782 non-null  int64  
 7   end_station_lat    1552782 non-null  float64
 8   end_station_lon    1552782 non-null  float64
 9   bike_id            1552782 non-null  object 
dtypes: float64(4), int64(3), object(3)
memory usage: 162.6+ MB


In [30]:
bike_set_clean['start_time'] = pd.to_datetime(bike_set_almost_clean['start_time'], format="%Y-%m-%d %H:%M:%S")
bike_set_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 0 to 1552781
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   Unnamed: 0         1552782 non-null  int64         
 1   start_time         1552782 non-null  datetime64[ns]
 2   end_time           1552782 non-null  object        
 3   start_station_id   1552782 non-null  int64         
 4   start_station_lat  1552782 non-null  float64       
 5   start_station_lon  1552782 non-null  float64       
 6   end_station_id     1552782 non-null  int64         
 7   end_station_lat    1552782 non-null  float64       
 8   end_station_lon    1552782 non-null  float64       
 9   bike_id            1552782 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(2)
memory usage: 162.6+ MB


In [31]:
bike_set_clean['end_time'] = pd.to_datetime(bike_set_almost_clean['end_time'], format="%Y-%m-%d %H:%M:%S")
bike_set_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 0 to 1552781
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   Unnamed: 0         1552782 non-null  int64         
 1   start_time         1552782 non-null  datetime64[ns]
 2   end_time           1552782 non-null  datetime64[ns]
 3   start_station_id   1552782 non-null  int64         
 4   start_station_lat  1552782 non-null  float64       
 5   start_station_lon  1552782 non-null  float64       
 6   end_station_id     1552782 non-null  int64         
 7   end_station_lat    1552782 non-null  float64       
 8   end_station_lon    1552782 non-null  float64       
 9   bike_id            1552782 non-null  object        
dtypes: datetime64[ns](2), float64(4), int64(3), object(1)
memory usage: 162.6+ MB


In [32]:
bike_set_clean.to_csv('outCSV/Clean_Metro_Set.csv')